In [1]:
import gc
import pickle
import numpy as np

import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint 
from tensorflow.keras.utils import Progbar
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import os
os.sys.path.append('../')

from BCNN import BCNN
from datasets import KaggleDR
from datasets import DatasetImageDataGenerator
# from training import generator_queue
# from util import Progplot

# go back to the root directory
os.chdir("../")

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
p = 0.2
last_layer = 'layer_17d'  # from JFnet
batch_size = 32
epochs = 30
#lr_schedule = {0: 0.005, 1: 0.005, 2: 0.001, 3: 0.001, 4: 0.0005, 5: 0.0001}
#change_every = 5
# we don't apply regularization to bias
l2_lambda = 0.001  # entire network
l1_lambda = 0.001  # only last layer
size = 512
n_classes = 2
dataset = 'KaggleDR'
seed = 1234

train_dir = "data/KaggleDR/train/"
test_dir = "data/KaggleDR/test/"

previous_weights = None

In [3]:
# paper uses train, test spit
"""
if dataset == 'KaggleDR':
    ds = KaggleDR(path_data='data/kaggle_dr/train_JF_BG_' + str(size),
                  filename_targets='data/kaggle_dr/trainLabels_01vs234.csv',
                  preprocessing=KaggleDR.standard_normalize,
                  require_both_eyes_same_label=False)
    ds_test = KaggleDR(path_data='data/kaggle_dr/test_JF_BG_' + str(size),
                       filename_targets='data/kaggle_dr/'
                                        'retinopathy_solution_01vs234.csv',
                       preprocessing=KaggleDR.standard_normalize,
                       require_both_eyes_same_label=False)
    idx_train, idx_val = train_test_split(np.arange(ds.n_samples),
                                          stratify=ds.y,
                                          test_size=0.2,
                                          random_state=seed)
"""
pass

In [4]:
# parameters for augmenting data
# Currently need to specify preprocessing function manually
AUGMENTATION_PARAMS = {'featurewise_center': False,
                       'samplewise_center': False,
                       'featurewise_std_normalization': False,
                       'samplewise_std_normalization': False,
                       'zca_whitening': False,
                       'rotation_range': 180.,
                       'width_shift_range': 0.05,
                       'height_shift_range': 0.05,
                       'shear_range': 0.,
                       'zoom_range': 0.10,
                       'channel_shift_range': 0.,
                       'fill_mode': 'constant',
                       'cval': 0.,
                       'horizontal_flip': True,
                       'vertical_flip': True,
                       #'dim_ordering': 'th'
                       'data_format' : 'channels_last',
                       # Preprocessing function ONLY FOR KAGGLE DR
                       'preprocessing_function' : KaggleDR.standard_normalize,
                       'validation_split':0.2,
                      }

train_datagen = ImageDataGenerator(**AUGMENTATION_PARAMS)

# create dataset using folder directories
# train feeds into augmenter
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(512, 512),
    batch_size=batch_size,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    subset='training'
)

# dataset for validation score
validation_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(512, 512),
    batch_size=batch_size,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    subset='validation'
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [5]:
# Setup networks
bcnn = BCNN(p_conv=p, last_layer=last_layer, n_classes=n_classes,
            l1_lambda=l1_lambda, l2_lambda=l2_lambda
           )
model = bcnn.net

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
l_out = model.layers[-1]

In [37]:
# create custom metrics (roc_auc)
#https://stackoverflow.com/questions/41032551/how-to-compute-receiving-operating-characteristic-roc-and-auc-in-keras
def auroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.float32)

In [41]:
# TODO: custom loss
def bayes_cross_entropy(y, ce_loss, n_classes):
    """Dalyac et al. (2014), eq. (17)"""
    # changed bincount to use numpy
    # need to shape to 1D from (len, 1)
    tf.cast(y, dtype=tf.int32)
    priors = tf.bincount(y) / y.shape[1]
    weights = 1.0 / (priors[y] * y.shape[1] * n_classes)
    bce_loss = ce_loss * weights
    return bce_loss.sum()

# custom loss function for the model
def bce_loss(n_classes):
    ce_loss = CategoricalCrossentropy()
    
    def loss(y_true, y_pred):
        ce = ce_loss(y_true, y_pred)
        return bayes_cross_entropy(y_true, ce, n_classes)
    
    return loss

In [42]:
bce_loss(n_classes)

<function __main__.bce_loss.<locals>.loss(y_true, y_pred)>

In [43]:
tf.bincount

<function tensorflow.python.ops.math_ops.bincount_v1(arr, weights=None, minlength=None, maxlength=None, dtype=tf.int32)>

In [44]:
model.compile(
    tf.keras.optimizers.Adam(), #trying different optimizer!
    loss=bce_loss(n_classes),
    metrics=[auroc]
)

ValueError: Tensor conversion requested dtype int32 for Tensor with dtype float32: 'Tensor("softmax_1_target_7:0", shape=(?, ?), dtype=float32)'

In [15]:
# Callbacks for training
callbacks = [
    ModelCheckpoint("models/new_bcnn.h5", monitor='val_loss', save_best_only=True, save_weights_only=True),
    CSVLogger("models/new_bccn_training.csv")
]

In [16]:
history = model.fit_generator(
    train_generator, 
    epochs=epochs, 
    verbose=1, 
    callbacks=callbacks, 
    validation_data=validation_generator,  
    workers=1, 
    use_multiprocessing=True
)

Epoch 1/30


IndexError: list index out of range

In [ ]:
pickle.dump(history, open('history' + last_layer + '.pkl', 'wb'))